In [ ]:
import SimpleITK as sitk
import numpy as np

#
# Essa função recebe uma imagem itk como entrada e 
# modifica a sua resolução. Se 
#
def resample_image(itk_image, out_spacing, is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    #
    # Observe aqui que se a imagem for do tipo Label (valores inteiros)
    # o interpolador usado é o vizinho mais próximo.
    #
    # O motivo para isso é que não é indesejável que a imagem (do tipo label)
    # possua valores de intensidade do tipo float após a interpolação
    # 
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)
#        resample.SetInterpolator(sitk.sitkNearestNeighbor)

    return resample.Execute(itk_image) 


# main program
reader = sitk.ImageFileReader()
reader.SetFileName('data\IXI002-Guys-0828-T1.nii.gz')
img = reader.Execute()

spacing_img = img.GetSpacing()


reader = sitk.ImageFileReader()
reader.SetFileName('atlas\mni_csf.nii.gz')
img_atlas = reader.Execute()

print("Informação imagem original")
print( img.GetSize() )
print( img.GetSpacing() )
print( img.GetDirection() )
print( img.GetOrigin() )

print("---------------")

spacing_atlas = img_atlas.GetSpacing()

img_resampled = resample_image(img_atlas, spacing_img)

print("Informação imagem reamostrada para ter voxels isotrópicos de 1mm")
print( img_resampled.GetSize() )
print( img_resampled.GetSpacing() )
print( img.GetDirection() )
print( img.GetOrigin() )
print("---------------")

writer = sitk.ImageFileWriter()
writer.SetFileName("test.nii")
writer.Execute(img_resampled)

In [ ]:
import SimpleITK as sitk
import numpy as np

# Função para reamostrar a imagem, ajustando resolução e tamanho
def resample_image(itk_image, out_spacing, out_size=None, is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    if out_size is None:
        # Calcula o novo tamanho baseado no espaçamento.
        out_size = [
            int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
            int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
            int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))
        ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    # Escolhe o interpolador com base no tipo da imagem
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)


# Programa principal
reader = sitk.ImageFileReader()
reader.SetFileName('data\\IXI002-Guys-0828-T1.nii.gz')
img = reader.Execute()

reader = sitk.ImageFileReader()
reader.SetFileName('atlas\\mni_csf.nii.gz')
img_atlas = reader.Execute()

print("Informação imagem original:")
print(f"Tamanho: {img.GetSize()}, Espaçamento: {img.GetSpacing()}")
print(f"Origin: {img.GetOrigin()}, Direction: {img.GetDirection()}")

print("Informação do atlas:")
print(f"Tamanho: {img_atlas.GetSize()}, Espaçamento: {img_atlas.GetSpacing()}")
print(f"Origin: {img_atlas.GetOrigin()}, Direction: {img_atlas.GetDirection()}")

# Define o espaçamento e o tamanho desejados para o atlas
spacing_img = img.GetSpacing()
size_img = img.GetSize()

# Reamostra o atlas para coincidir com o tamanho e espaçamento da imagem original
img_resampled = resample_image(img_atlas, out_spacing=spacing_img, out_size=size_img)

print("Informação imagem reamostrada:")
print(f"Tamanho: {img_resampled.GetSize()}, Espaçamento: {img_resampled.GetSpacing()}")
print(f"Origin: {img_resampled.GetOrigin()}, Direction: {img_resampled.GetDirection()}")

# Salva a imagem reamostrada
writer = sitk.ImageFileWriter()
writer.SetFileName("atlas_resampled_to_img.nii.gz")
writer.Execute(img_resampled)


In [19]:
import SimpleITK as sitk
import numpy as np

# Função para reamostrar a imagem, ajustando resolução, tamanho e propriedades
def resample_image(itk_image, out_spacing, out_size=None, reference_image=None, is_label=False):
    resample = sitk.ResampleImageFilter()

    # Define o tamanho e o espaçamento
    if reference_image is not None:
        # Usa as propriedades da imagem de referência
        resample.SetSize(reference_image.GetSize())
        resample.SetOutputSpacing(reference_image.GetSpacing())
        resample.SetOutputDirection(reference_image.GetDirection())
        resample.SetOutputOrigin(reference_image.GetOrigin())
    else:
        original_spacing = itk_image.GetSpacing()
        original_size = itk_image.GetSize()
        if out_size is None:
            # Calcula o novo tamanho baseado no espaçamento
            out_size = [
                int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
                int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
                int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))
            ]
        resample.SetSize(out_size)
        resample.SetOutputSpacing(out_spacing)
        resample.SetOutputDirection(itk_image.GetDirection())
        resample.SetOutputOrigin(itk_image.GetOrigin())

    # Configurações gerais do filtro
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    # Escolhe o interpolador com base no tipo da imagem
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)


# Programa principal
reader = sitk.ImageFileReader()
reader.SetFileName('data\\IXI002-Guys-0828-T1.nii.gz')
img = reader.Execute()

reader = sitk.ImageFileReader()
reader.SetFileName('atlas\\mni_csf.nii.gz')
img_atlas = reader.Execute()

print("Informação imagem original:")
print(f"Tamanho: {img.GetSize()}")
print(f"Espaçamento: {img.GetSpacing()}")
print(f"Origem: {img.GetOrigin()}")
print(f"Direção: {img.GetDirection()}")

print("Informação do atlas:")
print(f"Tamanho: {img_atlas.GetSize()}")
print(f"Espaçamento: {img_atlas.GetSpacing()}")
print(f"Origem: {img_atlas.GetOrigin()}")
print(f"Direção: {img_atlas.GetDirection()}")


# Reamostra o atlas para coincidir com a imagem original
img_resampled = resample_image(img_atlas, out_spacing=img.GetSpacing(), reference_image=img)

print("Informação imagem reamostrada:")
print(f"Tamanho: {img_resampled.GetSize()}")
print(f"Espaçamento: {img_resampled.GetSpacing()}")
print(f"Origem: {img_resampled.GetOrigin()}")
print(f"Direção: {img_resampled.GetDirection()}")


# Salva a imagem reamostrada
writer = sitk.ImageFileWriter()
writer.SetFileName("atlas_resampled_to_img.nii.gz")
writer.Execute(img_resampled)


Informação imagem original:
Tamanho: (256, 256, 150)
Espaçamento: (0.9375, 0.9375, 1.1999969482421875)
Origem: (88.639892578125, -116.5320053100586, -112.11355590820312)
Direção: (-0.0, 0.0, -1.0, 0.9923758134408762, -0.12324871154522958, 0.0, 0.12324871154522958, 0.9923758134408762, -2.0816681711721685e-16)
Informação do atlas:
Tamanho: (256, 256, 256)
Espaçamento: (1.0, 1.0, 1.0)
Origem: (-127.5, -127.5, -127.5)
Direção: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
Informação imagem reamostrada:
Tamanho: (256, 256, 150)
Espaçamento: (0.9375, 0.9375, 1.1999969482421875)
Origem: (88.639892578125, -116.5320053100586, -112.11355590820312)
Direção: (-0.0, 0.0, -1.0, 0.9923758134408762, -0.12324871154522958, 0.0, 0.12324871154522958, 0.9923758134408762, -2.0816681711721685e-16)


In [20]:
reader = sitk.ImageFileReader()
reader.SetFileName('atlas_resampled_to_img.nii.gz')
img_atlas = reader.Execute()

print("Informação imagem Transformada:")
print(f"Tamanho: {img.GetSize()}")
print(f"Espaçamento: {img.GetSpacing()}")
print(f"Origem: {img.GetOrigin()}")
print(f"Direção: {img.GetDirection()}")

Informação imagem Transformada:
Tamanho: (256, 256, 150)
Espaçamento: (0.9375, 0.9375, 1.1999969482421875)
Origem: (88.639892578125, -116.5320053100586, -112.11355590820312)
Direção: (-0.0, 0.0, -1.0, 0.9923758134408762, -0.12324871154522958, 0.0, 0.12324871154522958, 0.9923758134408762, -2.0816681711721685e-16)
